In [56]:
# Importing packages
import nltk
import numpy as np
import tensorflow as tf
from distutils.version import LooseVersion
import warnings
from tensorflow.python.layers.core import Dense
import pickle
import os
import copy
import re
import unicodedata
from collections import Counter
import random

# https://github.com/deep-diver/EN-FR-MLT-tensorflow/blob/master/dlnd_language_translationv2.ipynb

In [ ]:
# Lets see if we can preprocess it like the example
f=open('PrideandPred.txt', encoding = 'utf8')
st2 = nltk.data.load('tokenizers/punkt/english.pickle')
raw=f.read()
raw = raw.replace('\ufeff','').replace('\n',' ').replace('  ',' ').replace('_','')
for i in range(0,100):
    raw = raw.replace('Chapter '+str(i)+' ','')
raw = re.sub('([.,!?()])', r' \1 ', raw)
raw = re.sub('\s{2,}', ' ', raw)
raw = raw.lower()
#raw = '\n'.join(st2.tokenize(raw, realign_boundaries=True))
raw = st2.tokenize(raw, realign_boundaries=True)
print(*raw, sep='\n\n')
# Lets hold off on Pride and Prejudice for now, using the example text

In [89]:
f = open('small_vocab_en.txt')
raw = f.read()
print(len(raw.split('\n')))
print(len(set(raw.split())))

137861
227


In [90]:
# Okay... lets try movie lines instead
f=open('movie_lines.txt', encoding = 'utf-8', errors='replace')
raw=f.read()
raw = raw.split('\n')
for i in ['.',',','!','?',';',':','"']:
    raw = [x.replace(i,' '+i+' ') for x in raw]
raw = [x.replace('. . .','...') for x in raw]
raw = [' '.join(x.split()) for x in raw]
raw = [x.split('+++$+++')[-1] for x in raw]
#raw = [x.replace(' . ', ' . \n') for x in raw]
raw = [x.lstrip() for x in raw]
raw = '\n'.join([x for x in raw if len(x)>10])
raw = unicodedata.normalize('NFKD', raw)
raw = '\n'.join(random.sample(raw.split('\n'),10000))
print(len(set(raw.split())))
print(raw[0:1000])

12797
Thank you .
-- You remember .
They're not going to listen to me . You either . Are you kidding ?
It's only temporary . On a freelance basis .
Rusty's dead . That's Ajax
That's what I was hoping you'd tell me .
I'll call you Mark when you catch the killer , Detective .
Precisely , Maria . Tonight you have the same privilege that comes on rare occasions to the chief executive of some state or nation . . . the privilege of restoring life , by one tender act of mercy , to a doomed fellow creature .
I can feel air .
Uh , yoo-hoo . Excuse me ! Sorry to interrupt , but I got some big news .
They don't have substitute teachers where you go to school ?
It probably will . In fact , I'd go so far as to say it almost certainly will , in time . Why should I settle for that ?
He says it's a box .
I'm thinking about producing .
There is a motel downtown , near the Machine Shop . . . the Suncrest . Room 138 .
Working on it now , sir .
There is a 20th Century possibility .
What ? What is it ?
Ben

In [91]:
# Saving as a text file
file = open('ProcessedScript.txt', 'w', encoding = 'utf-8', errors = 'replace') 
file.write(raw)
file.close() 

# Can we read it back in
raw = open('ProcessedScript.txt', 'r', encoding = 'utf-8', errors = 'replace')
raw = raw.read()
print(raw[0:1000])
print(len(raw.split('\n')))

Thank you .
-- You remember .
They're not going to listen to me . You either . Are you kidding ?
It's only temporary . On a freelance basis .
Rusty's dead . That's Ajax
That's what I was hoping you'd tell me .
I'll call you Mark when you catch the killer , Detective .
Precisely , Maria . Tonight you have the same privilege that comes on rare occasions to the chief executive of some state or nation . . . the privilege of restoring life , by one tender act of mercy , to a doomed fellow creature .
I can feel air .
Uh , yoo-hoo . Excuse me ! Sorry to interrupt , but I got some big news .
They don't have substitute teachers where you go to school ?
It probably will . In fact , I'd go so far as to say it almost certainly will , in time . Why should I settle for that ?
He says it's a box .
I'm thinking about producing .
There is a motel downtown , near the Machine Shop . . . the Suncrest . Room 138 .
Working on it now , sir .
There is a 20th Century possibility .
What ? What is it ?
Ben is ve

In [92]:
# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.1'), 'Please use TensorFlow version 1.1 or newer'
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.12.0
Default GPU Device: /device:GPU:0


In [122]:
# Preprocessing and Preparing data
def load_data(path, vocab_size):
    input_file = os.path.join(path)
    with open(input_file, 'r', encoding='utf-8', errors='replace') as f:
        data = f.read()
        #data = data.replace('\ufeff','').replace('\n',' ').replace('  ',' ').replace('_','')
        #for i in range(0,100):
        #    data = data.replace('Chapter '+str(i)+' ','')
        #data = ' '.join(nltk.tokenize.sent_tokenize(data)[0:20000])
    if vocab_size != None:
        top_n = [x[0] for x in Counter(data.split()).most_common(vocab_size)]
        data = ' '.join([x if x in top_n else '<UNK>' for x in data.split()])
        data = '\n'.join([x for x in data.split('\n') if len(set(x.split()))>1])

    return data

CODES = {'<PAD>': 0, '<EOS>': 1, '<UNK>': 2, '<GO>': 3 }

def create_lookup_tables(text):
    # make a list of unique words
    vocab = set(text.split())
    #vocab = set(nltk.tokenize.word_tokenize(text))

    # (1)
    # starts with the special tokens
    vocab_to_int = copy.copy(CODES)

    # the index (v_i) will starts from 4 (the 2nd arg in enumerate() specifies the starting index)
    # since vocab_to_int already contains special tokens
    for v_i, v in enumerate(vocab, len(CODES)):
        vocab_to_int[v] = v_i

    # (2)
    int_to_vocab = {v_i: v for v, v_i in vocab_to_int.items()}

    return vocab_to_int, int_to_vocab

def text_to_ids(source_text, target_text, source_vocab_to_int, target_vocab_to_int):
    """
        1st, 2nd args: raw string text to be converted
        3rd, 4th args: lookup tables for 1st and 2nd args respectively
    
        return: A tuple of lists (source_id_text, target_id_text) converted
    """
    # empty list of converted sentences
    source_text_id = []
    target_text_id = []
    
    # make a list of sentences (extraction)
    source_sentences = source_text.split("\n")
    target_sentences = target_text.split("\n")
    #source_sentences = nltk.tokenize.sent_tokenize(source_text)
    #target_sentences = nltk.tokenize.sent_tokenize(target_text)
    
    max_source_sentence_length = max([len(sentence.split(" ")) for sentence in source_sentences])
    max_target_sentence_length = max([len(sentence.split(" ")) for sentence in target_sentences])
    
    # iterating through each sentences (# of sentences in source&target is the same)
    for i in range(len(source_sentences)):
        # extract sentences one by one
        source_sentence = source_sentences[i]
        target_sentence = target_sentences[i]
        
        # Putting whitespace around punctuation
        #source_sentence = re.sub('([.,!?()])', r' \1 ', source_sentence)
        #source_sentence = re.sub('\s{2,}', ' ', source_sentence)
        #target_sentence = re.sub('([.,!?()])', r' \1 ', target_sentence)
        #target_sentence = re.sub('\s{2,}', ' ', target_sentence)
        
        # make a list of tokens/words (extraction) from the chosen sentence
        source_tokens = source_sentence.split(" ")
        target_tokens = target_sentence.split(" ")
        #source_tokens = nltk.tokenize.word_tokenize(source_sentence)
        #target_tokens = nltk.tokenize.word_tokenize(target_sentence)
        
        # empty list of converted words to index in the chosen sentence
        source_token_id = []
        target_token_id = []
        
        for index, token in enumerate(source_tokens):
            if (token != ""):
                source_token_id.append(source_vocab_to_int[token])
        
        for index, token in enumerate(target_tokens):
            if (token != ""):
                target_token_id.append(target_vocab_to_int[token])
                
        # put <EOS> token at the end of the chosen target sentence
        # this token suggests when to stop creating a sequence
        target_token_id.append(target_vocab_to_int['<EOS>'])
            
        # add each converted sentences in the final list
        source_text_id.append(source_token_id)
        target_text_id.append(target_token_id)
    
    return source_text_id, target_text_id


def preprocess_and_save_data(source_path, target_path, text_to_ids, max_vocab=None):
    # Preprocess
    
    # load original data (English, French)
    source_text = load_data(source_path, max_vocab)
    target_text = load_data(target_path, max_vocab)

    # to the lower case
    source_text = source_text.lower()
    target_text = target_text.lower()

    # create lookup tables for data
    source_vocab_to_int, source_int_to_vocab = create_lookup_tables(source_text)
    target_vocab_to_int, target_int_to_vocab = create_lookup_tables(target_text)

    # create list of sentences whose words are represented in index
    source_text, target_text = text_to_ids(source_text, target_text, source_vocab_to_int, target_vocab_to_int)

    # Save data for later use
    pickle.dump((
        (source_text, target_text),
        (source_vocab_to_int, target_vocab_to_int),
        (source_int_to_vocab, target_int_to_vocab)), open('preprocess.p', 'wb'))
    
source_path = 'small_vocab_en.txt'
target_path = 'small_vocab_en.txt'
preprocess_and_save_data(source_path, target_path, text_to_ids, max_vocab=None)

In [123]:
def load_preprocess():
    with open('preprocess.p', mode='rb') as in_file:
        return pickle.load(in_file)
    
(source_int_text, target_int_text), (source_vocab_to_int, target_vocab_to_int), _ = load_preprocess()

In [124]:
def enc_dec_model_inputs():
    inputs = tf.placeholder(tf.int32, [None, None], name='input')
    targets = tf.placeholder(tf.int32, [None, None], name='targets') 
    
    target_sequence_length = tf.placeholder(tf.int32, [None], name='target_sequence_length')
    max_target_len = tf.reduce_max(target_sequence_length)    
    
    return inputs, targets, target_sequence_length, max_target_len

In [125]:
def hyperparam_inputs():
    lr_rate = tf.placeholder(tf.float32, name='lr_rate')
    keep_prob = tf.placeholder(tf.float32, name='keep_prob')
    
    return lr_rate, keep_prob

In [126]:
def process_decoder_input(target_data, target_vocab_to_int, batch_size):
    """
    Preprocess target data for encoding
    :return: Preprocessed target data
    """
    # get '<GO>' id
    go_id = target_vocab_to_int['<GO>']
    
    after_slice = tf.strided_slice(target_data, [0, 0], [batch_size, -1], [1, 1])
    after_concat = tf.concat( [tf.fill([batch_size, 1], go_id), after_slice], 1)
    
    return after_concat

In [127]:
def encoding_layer(rnn_inputs, rnn_size, num_layers, keep_prob, 
                   source_vocab_size, 
                   encoding_embedding_size):
    """
    :return: tuple (RNN output, RNN state)
    """
    embed = tf.contrib.layers.embed_sequence(rnn_inputs, 
                                             vocab_size=source_vocab_size, 
                                             embed_dim=encoding_embedding_size)
    
    stacked_cells = tf.contrib.rnn.MultiRNNCell([tf.contrib.rnn.DropoutWrapper(tf.contrib.rnn.LSTMCell(rnn_size), keep_prob) for _ in range(num_layers)])
    
    outputs, state = tf.nn.dynamic_rnn(stacked_cells, 
                                       embed, 
                                       dtype=tf.float32)
    return outputs, state

In [128]:
def decoding_layer_train(encoder_state, dec_cell, dec_embed_input, 
                         target_sequence_length, max_summary_length, 
                         output_layer, keep_prob):
    """
    Create a training process in decoding layer 
    :return: BasicDecoderOutput containing training logits and sample_id
    """
    dec_cell = tf.contrib.rnn.DropoutWrapper(dec_cell, 
                                             output_keep_prob=keep_prob)
    
    # for only input layer
    helper = tf.contrib.seq2seq.TrainingHelper(dec_embed_input, 
                                               target_sequence_length)
    
    decoder = tf.contrib.seq2seq.BasicDecoder(dec_cell, 
                                              helper, 
                                              encoder_state, 
                                              output_layer)

    # unrolling the decoder layer
    outputs, _, _ = tf.contrib.seq2seq.dynamic_decode(decoder, 
                                                      impute_finished=True, 
                                                      maximum_iterations=max_summary_length)
    return outputs

In [129]:
def decoding_layer_infer(encoder_state, dec_cell, dec_embeddings, start_of_sequence_id,
                         end_of_sequence_id, max_target_sequence_length,
                         vocab_size, output_layer, batch_size, keep_prob):
    """
    Create a inference process in decoding layer 
    :return: BasicDecoderOutput containing inference logits and sample_id
    """
    dec_cell = tf.contrib.rnn.DropoutWrapper(dec_cell, 
                                             output_keep_prob=keep_prob)
    
    helper = tf.contrib.seq2seq.GreedyEmbeddingHelper(dec_embeddings, 
                                                      tf.fill([batch_size], start_of_sequence_id), 
                                                      end_of_sequence_id)
    
    decoder = tf.contrib.seq2seq.BasicDecoder(dec_cell, 
                                              helper, 
                                              encoder_state, 
                                              output_layer)
    
    outputs, _, _ = tf.contrib.seq2seq.dynamic_decode(decoder, 
                                                      impute_finished=True, 
                                                      maximum_iterations=max_target_sequence_length)
    return outputs

In [130]:
def decoding_layer(dec_input, encoder_state,
                   target_sequence_length, max_target_sequence_length,
                   rnn_size,
                   num_layers, target_vocab_to_int, target_vocab_size,
                   batch_size, keep_prob, decoding_embedding_size):
    """
    Create decoding layer
    :return: Tuple of (Training BasicDecoderOutput, Inference BasicDecoderOutput)
    """
    target_vocab_size = len(target_vocab_to_int)
    dec_embeddings = tf.Variable(tf.random_uniform([target_vocab_size, decoding_embedding_size]))
    dec_embed_input = tf.nn.embedding_lookup(dec_embeddings, dec_input)
    
    cells = tf.contrib.rnn.MultiRNNCell([tf.contrib.rnn.LSTMCell(rnn_size) for _ in range(num_layers)])
    
    with tf.variable_scope("decode"):
        output_layer = tf.layers.Dense(target_vocab_size)
        train_output = decoding_layer_train(encoder_state, 
                                            cells, 
                                            dec_embed_input, 
                                            target_sequence_length, 
                                            max_target_sequence_length, 
                                            output_layer, 
                                            keep_prob)

    with tf.variable_scope("decode", reuse=True):
        infer_output = decoding_layer_infer(encoder_state, 
                                            cells, 
                                            dec_embeddings, 
                                            target_vocab_to_int['<GO>'], 
                                            target_vocab_to_int['<EOS>'], 
                                            max_target_sequence_length, 
                                            target_vocab_size, 
                                            output_layer,
                                            batch_size,
                                            keep_prob)

    return (train_output, infer_output)

In [131]:
def seq2seq_model(input_data, target_data, keep_prob, batch_size,
                  target_sequence_length,
                  max_target_sentence_length,
                  source_vocab_size, target_vocab_size,
                  enc_embedding_size, dec_embedding_size,
                  rnn_size, num_layers, target_vocab_to_int):
    """
    Build the Sequence-to-Sequence model
    :return: Tuple of (Training BasicDecoderOutput, Inference BasicDecoderOutput)
    """
    enc_outputs, enc_states = encoding_layer(input_data, 
                                             rnn_size, 
                                             num_layers, 
                                             keep_prob, 
                                             source_vocab_size, 
                                             enc_embedding_size)
    
    dec_input = process_decoder_input(target_data, 
                                      target_vocab_to_int, 
                                      batch_size)
    
    train_output, infer_output = decoding_layer(dec_input,
                                               enc_states, 
                                               target_sequence_length, 
                                               max_target_sentence_length,
                                               rnn_size,
                                              num_layers,
                                              target_vocab_to_int,
                                              target_vocab_size,
                                              batch_size,
                                              keep_prob,
                                              dec_embedding_size)
    
    return train_output, infer_output

In [140]:
display_step = 300

epochs = 5
batch_size = 64

rnn_size = 128
num_layers = 3

encoding_embedding_size = 200
decoding_embedding_size = 200

learning_rate = 0.001
keep_probability = 0.5

In [141]:
save_path = 'checkpoints/dev'
(source_int_text, target_int_text), (source_vocab_to_int, target_vocab_to_int), _ = load_preprocess()
max_target_sentence_length = max([len(sentence) for sentence in source_int_text])

train_graph = tf.Graph()
with train_graph.as_default():
    input_data, targets, target_sequence_length, max_target_sequence_length = enc_dec_model_inputs()
    lr, keep_prob = hyperparam_inputs()
    
    train_logits, inference_logits = seq2seq_model(tf.reverse(input_data, [-1]),
                                                   targets,
                                                   keep_prob,
                                                   batch_size,
                                                   target_sequence_length,
                                                   max_target_sequence_length,
                                                   len(source_vocab_to_int),
                                                   len(target_vocab_to_int),
                                                   encoding_embedding_size,
                                                   decoding_embedding_size,
                                                   rnn_size,
                                                   num_layers,
                                                   target_vocab_to_int)
    
    training_logits = tf.identity(train_logits.rnn_output, name='logits')
    inference_logits = tf.identity(inference_logits.sample_id, name='predictions')

    # https://www.tensorflow.org/api_docs/python/tf/sequence_mask
    # - Returns a mask tensor representing the first N positions of each cell.
    masks = tf.sequence_mask(target_sequence_length, max_target_sequence_length, dtype=tf.float32, name='masks')

    with tf.name_scope("optimization"):
        # Loss function - weighted softmax cross entropy
        cost = tf.contrib.seq2seq.sequence_loss(
            training_logits,
            targets,
            masks)

        # Optimizer
        optimizer = tf.train.AdamOptimizer(lr)

        # Gradient Clipping
        gradients = optimizer.compute_gradients(cost)
        capped_gradients = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gradients if grad is not None]
        train_op = optimizer.apply_gradients(capped_gradients)

In [142]:
def pad_sentence_batch(sentence_batch, pad_int):
    """Pad sentences with <PAD> so that each sentence of a batch has the same length"""
    max_sentence = max([len(sentence) for sentence in sentence_batch])
    return [sentence + [pad_int] * (max_sentence - len(sentence)) for sentence in sentence_batch]


def get_batches(sources, targets, batch_size, source_pad_int, target_pad_int):
    """Batch targets, sources, and the lengths of their sentences together"""
    for batch_i in range(0, len(sources)//batch_size):
        start_i = batch_i * batch_size

        # Slice the right amount for the batch
        sources_batch = sources[start_i:start_i + batch_size]
        targets_batch = targets[start_i:start_i + batch_size]

        # Pad
        pad_sources_batch = np.array(pad_sentence_batch(sources_batch, source_pad_int))
        pad_targets_batch = np.array(pad_sentence_batch(targets_batch, target_pad_int))

        # Need the lengths for the _lengths parameters
        pad_targets_lengths = []
        for target in pad_targets_batch:
            pad_targets_lengths.append(len(target))

        pad_source_lengths = []
        for source in pad_sources_batch:
            pad_source_lengths.append(len(source))

        yield pad_sources_batch, pad_targets_batch, pad_source_lengths, pad_targets_lengths

In [143]:
def get_accuracy(target, logits):
    """
    Calculate accuracy
    """
    max_seq = max(target.shape[1], logits.shape[1])
    if max_seq - target.shape[1]:
        target = np.pad(
            target,
            [(0,0),(0,max_seq - target.shape[1])],
            'constant')
    if max_seq - logits.shape[1]:
        logits = np.pad(
            logits,
            [(0,0),(0,max_seq - logits.shape[1])],
            'constant')

    return np.mean(np.equal(target, logits))

# Split data to training and validation sets
train_source = source_int_text[batch_size:]
train_target = target_int_text[batch_size:]
valid_source = source_int_text[:batch_size]
valid_target = target_int_text[:batch_size]
(valid_sources_batch, valid_targets_batch, valid_sources_lengths, valid_targets_lengths ) = next(get_batches(valid_source,
                                                                                                             valid_target,
                                                                                                             batch_size,
                                                                                                             source_vocab_to_int['<PAD>'],
                                                                                                             target_vocab_to_int['<PAD>']))                                                                                                  
print('Done splitting into train and val!')
with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())
    print('Starting seq2seq training...')
    for epoch_i in range(epochs):
        for batch_i, (source_batch, target_batch, sources_lengths, targets_lengths) in enumerate(
                get_batches(train_source, train_target, batch_size,
                            source_vocab_to_int['<PAD>'],
                            target_vocab_to_int['<PAD>'])):

            _, loss = sess.run(
                [train_op, cost],
                {input_data: source_batch,
                 targets: target_batch,
                 lr: learning_rate,
                 target_sequence_length: targets_lengths,
                 keep_prob: keep_probability})


            if batch_i % display_step == 0 and batch_i > 0:
                batch_train_logits = sess.run(
                    inference_logits,
                    {input_data: source_batch,
                     target_sequence_length: targets_lengths,
                     keep_prob: 1.0})

                batch_valid_logits = sess.run(
                    inference_logits,
                    {input_data: valid_sources_batch,
                     target_sequence_length: valid_targets_lengths,
                     keep_prob: 1.0})

                train_acc = get_accuracy(target_batch, batch_train_logits)
                valid_acc = get_accuracy(valid_targets_batch, batch_valid_logits)
    
                print('Epoch {:>3} Batch {:>4}/{} - Train Accuracy: {:>6.4f}, Validation Accuracy: {:>6.4f}, Loss: {:>6.4f}'
                      .format(epoch_i+1, batch_i, len(source_int_text) // batch_size, train_acc, valid_acc, loss))

    # Save Model
    saver = tf.train.Saver()
    saver.save(sess, save_path)
    print('Model Trained and Saved')

Done splitting into train and val!
Starting seq2seq training...
Epoch   1 Batch  300/2154 - Train Accuracy: 0.6011, Validation Accuracy: 0.5859, Loss: 1.6724
Epoch   1 Batch  600/2154 - Train Accuracy: 0.6780, Validation Accuracy: 0.6762, Loss: 1.0294
Epoch   1 Batch  900/2154 - Train Accuracy: 0.7283, Validation Accuracy: 0.7170, Loss: 0.8420
Epoch   1 Batch 1200/2154 - Train Accuracy: 0.7448, Validation Accuracy: 0.7344, Loss: 0.7366
Epoch   1 Batch 1500/2154 - Train Accuracy: 0.7517, Validation Accuracy: 0.7465, Loss: 0.6965
Epoch   1 Batch 1800/2154 - Train Accuracy: 0.7474, Validation Accuracy: 0.7474, Loss: 0.6946
Epoch   1 Batch 2100/2154 - Train Accuracy: 0.7648, Validation Accuracy: 0.7526, Loss: 0.6409
Epoch   2 Batch  300/2154 - Train Accuracy: 0.7518, Validation Accuracy: 0.7691, Loss: 0.6751
Epoch   2 Batch  600/2154 - Train Accuracy: 0.8177, Validation Accuracy: 0.8012, Loss: 0.5752
Epoch   2 Batch  900/2154 - Train Accuracy: 0.8212, Validation Accuracy: 0.8125, Loss: 0.5

In [144]:
def save_params(params):
    with open('params.p', 'wb') as out_file:
        pickle.dump(params, out_file)


def load_params():
    with open('params.p', mode='rb') as in_file:
        return pickle.load(in_file)

In [145]:
# Save parameters for checkpoint
save_params(save_path)

In [146]:
import tensorflow as tf
import numpy as np

_, (source_vocab_to_int, target_vocab_to_int), (source_int_to_vocab, target_int_to_vocab) = load_preprocess()
load_path = load_params()

In [150]:
def sentence_to_seq(sentence, vocab_to_int):
    results = []
    for word in sentence.split(" "):
        if word in vocab_to_int:
            results.append(vocab_to_int[word])
        else:
            results.append(vocab_to_int['<UNK>'])
            
    return results

translate_sentence = 'lemons are your least favorite fruit .'

translate_sentence = sentence_to_seq(translate_sentence, source_vocab_to_int)

loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.train.import_meta_graph(load_path + '.meta')
    loader.restore(sess, load_path)

    input_data = loaded_graph.get_tensor_by_name('input:0')
    logits = loaded_graph.get_tensor_by_name('predictions:0')
    target_sequence_length = loaded_graph.get_tensor_by_name('target_sequence_length:0')
    keep_prob = loaded_graph.get_tensor_by_name('keep_prob:0')

    translate_logits = sess.run(logits, {input_data: [translate_sentence]*batch_size,
                                         target_sequence_length: [len(translate_sentence)*2]*batch_size,
                                         keep_prob: 1.0})[0]

print('Input')
print('  Word Ids:      {}'.format([i for i in translate_sentence]))
print('  English Words: {}'.format(" ".join([source_int_to_vocab[i] for i in translate_sentence])))

print('\nPrediction')
print('  Word Ids:      {}'.format([i for i in translate_logits]))
print('  Reconstruc. Words: {}'.format(" ".join([target_int_to_vocab[i] for i in translate_logits])))

INFO:tensorflow:Restoring parameters from checkpoints/dev
Input
  Word Ids:      [158, 50, 46, 196, 114, 182, 69]
  English Words: lemons are your least favorite fruit .

Prediction
  Word Ids:      [204, 114, 182, 190, 180, 89, 190, 180, 20, 69, 1]
  Reconstruc. Words: their favorite fruit is the banana is the pear . <EOS>
